In [19]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Lambda, Dense
from keras.models import Model
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras import backend as K
from sklearn.model_selection import train_test_split

In [20]:
# Đường dẫn đến dữ liệu
PAIRS_PATH = "data/pairs/pairs.npy"
LABELS_PATH = "data/pairs/labels.npy"

In [21]:
# Tải dữ liệu cặp ảnh và nhãn
pairs = np.load(PAIRS_PATH)
labels = np.load(LABELS_PATH)

In [22]:
# Chia dữ liệu thành train/test
pairs_train, pairs_test, labels_train, labels_test = train_test_split(
    pairs, labels, test_size=0.2, random_state=42
)


In [23]:
# Kích thước đầu vào
input_shape = (160, 160, 3)

In [24]:
# Base Network: ResNet50
def create_base_network(input_shape):
    """
    Tạo mạng con ResNet50 để trích xuất đặc trưng.
    """
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape, pooling="avg")
    return Model(inputs=base_model.input, outputs=base_model.output)

In [ ]:
# Siamese Network
def create_siamese_network(input_shape):
    """
    Xây dựng mô hình Siamese Network.
    """
    base_network = create_base_network(input_shape)

    # Hai đầu vào
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # Trích xuất đặc trưng từ Base Network
    embedding_a = base_network(input_a)
    embedding_b = base_network(input_b)

# Lớp Lambda để tính khoảng cách Euclidean
    def euclidean_distance(vectors):
        x, y = vectors
        return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

    distance = Lambda(euclidean_distance, name="Euclidean_Distance")([embedding_a, embedding_b])

    # Lớp đầu ra (binary classification)
    output = Dense(1, activation="sigmoid", name="Output")(distance)

    # Mô hình hoàn chỉnh
    siamese_model = Model(inputs=[input_a, input_b], outputs=output)
    return siamese_model


# Hàm mất mát Contrastive Loss
def contrastive_loss(y_true, y_pred):
    """
    Hàm mất mát Contrastive Loss.
    Args:
        y_true: Nhãn thực tế (0 hoặc 1).
        y_pred: Khoảng cách Euclidean giữa các embeddings.
    Returns:
        Loss value.
    """
    margin = 1.0
    # Chuyển đổi y_true sang float32
    y_true = K.cast(y_true, K.floatx())
    return K.mean(
        y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0))
    )

In [26]:
# Biên dịch mô hình
siamese_model = create_siamese_network(input_shape)
siamese_model.compile(loss=contrastive_loss, optimizer=Adam(learning_rate=0.0001), metrics=["accuracy"])
siamese_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_6 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_2 (Functional)           (None, 2048)         23587712    ['input_5[0][0]',                
                                                                  'input_6[0][0]']          

In [27]:

# Hàm load và chuẩn hóa ảnh
def load_and_preprocess_image(image_path):
    """
    Tải và chuẩn hóa ảnh.
    """
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

In [28]:
# Chuẩn bị dữ liệu
def generate_data(pairs, labels, batch_size=32):
    """
    Generator để tạo dữ liệu batch cho huấn luyện.
    """
    while True:
        for i in range(0, len(pairs), batch_size):
            batch_pairs = pairs[i : i + batch_size]
            batch_labels = labels[i : i + batch_size]

            images_a = np.array([load_and_preprocess_image(pair[0]) for pair in batch_pairs])
            images_b = np.array([load_and_preprocess_image(pair[1]) for pair in batch_pairs])

            yield [images_a, images_b], np.array(batch_labels)

In [29]:
# Huấn luyện mô hình
batch_size = 32
epochs = 10

train_gen = generate_data(pairs_train, labels_train, batch_size)
test_gen = generate_data(pairs_test, labels_test, batch_size)

steps_per_epoch = len(pairs_train) // batch_size
validation_steps = len(pairs_test) // batch_size

siamese_model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_gen,
    validation_steps=validation_steps,
    epochs=epochs,
)

# Lưu mô hình
siamese_model.save("models/siamese_resnet_model.h5")
print("Mô hình đã được lưu.")

Epoch 1/10


TypeError: in user code:

    File "c:\Users\HUY THONG\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\HUY THONG\AppData\Local\Temp\ipykernel_21940\2491474645.py", line 37, in contrastive_loss  *
        y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0))

    TypeError: Input 'y' of 'Mul' Op has type float32 that does not match type int32 of argument 'x'.
